In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

In [ ]:
# sklearn 1.0 버전 부터 boston 주택가격 데이터셋이 삭제되었다.
# 다음과 같이 boston 데이터셋을 불러온다.
from sklearn.datasets import fetch_openml
boston = fetch_openml(name='boston')

df = pd.DataFrame(boston.data, columns=boston.feature_names)
df["TARGET"] = boston.target
df.tail()

In [ ]:
# 표준 스케일링(standard scaling)을 통해 입력 값을 정규화한다.
scaler = StandardScaler()
scaler.fit(df.values[:, :-1])
df.values[:, :-1] = scaler.transform(df.values[:, :-1]).round(4)

df.tail()

In [ ]:
# 정규화를 하는 이유
# 적절한 정규화는 신경망의 최적화를 수월하게 만들 수 있다.
# 정규화를 적용하기 전, 데이터셋 분포의 특징을 파악하고 어떤 정규화 방법이 가장 어울릴지 결정한다.
# 위 예제에선, 보스턴 주택가격 데이터셋의 각 열이 정규분포(normal distribution)을 따른다고 가정하고, 표준 스케일링을 적용한다.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# 학습에 필요한 패키지를 불러오고, 판다스에 저장된 넘파이 값을 파이토치 텐서로 변환하여 입력 텐서 x와 출력 텐서 y를 만든다.
data = torch.from_numpy(df.values.astype('float32')) 

y = data[:, -1:]
x = data[:, :-1]

print(x.shape, y.shape)

In [ ]:
# 학습에 필요한 설정값을 정한다.
# 이 설정값을 바꿔가며 학습하면, 더 많은 경험을 쌓을 수 있다.
n_epochs = 200000
learning_rate = 1e-4
print_interval = 10000

In [ ]:
# 이제 심층신경망을 정의한다.
# nn.Module을 상속받아 MyModule이라는 나만의 모델 클래스를 정의한다.
# 심층 신경망은 4개의 선형 계층과 비선형 함수를 갖도록 한다.
'''
__init 함수를 보면 선형 계층은 각각 linear1, linear2, linear3, linear4라는 이름ㅇ르 갖도록 선언한다.
비선형 활성화 함수는 ReLU를 사용한다.
다만, 선형 계층들은 각각 다른 가중치 파라미터를 가지게 되므로 다른 객체로 선언한다.
비선형 활성 함수의 경우, 학습되는 파라미터를 갖지 않기에 모든 계층에서 동일하게 동작하므로, 한 개만 선언하여 재활용하도록 한다.
각 선형 계층의 입출력 크기는 최초 입력 차원(input_dim)과 최종 출력 차원(output_dim)을 제외하고는 임의로 정해준다.

forward 함수에서는 앞서 선언된 내부 모듈들을 활용하여, 피드포워드(feed-forward) 연산을 수행할 수 있도록 한다.
x라는 샘플 개수 곱하기 입력 차원(batch_size, input_dim) 크기의 2차원 텐서가 주어지면, 최종적으로 샘플 개수 곱하기 출력 차원(batch_size, output_dim) 크기의 2차원 텐서로 뱉어내는 함수가 된다.
input_dim과 output_dim은 __init__ 함수에서 미리 입력받는 것을 볼 수 있다.

마지막 계층에서는 활성 함수를 씌우지 않도록 주의해야 한다.
'''

class MyModule(nn.Module):

    def __init__(self, input_dim, output_dim):
        self.input_dim = input_dim
        self.output_dim = output_dim

        super().__init__()

        self.linear1 = nn.Linear(input_dim, 3)
        self.linear2 = nn.Linear(3, 3)
        self.linear3 = nn.Linear(3, 3)
        self.linear4 = nn.Linear(3, output_dim)
        self.act = nn.ReLU()
    
    def forward(self, x):
        # |x| = (batch_size, input_dim)
        h = self.act(self.linear1(x)) # |h| = (batch_size, 3)
        h = self.act(self.linear2(h))
        h = self.act(self.linear3(h))
        y = self.linear4(h)
        # |y| = (batch_size, output_dim)

        return y

model = MyModule(x.size(-1), y.size(-1))

print(model)

In [ ]:
# 지금은 모델 구조가 매우 단순한 편이다.
# 입력 텐서를 받아 단순하게 순차적으로 앞으로 하나씩 계산해 나가는 것에 불과하기 때문이다.
# 이 경우, 나만의 모델 클래스를 정의하는 대신 다음과 같이 nn.Sequential 클래스를 활용하여, 훨씬 쉽게 모델 객체를 선언할 수 있다.
# 다음은 앞의 MyModel 클래스를 똑같은 구조를 갖는 심층신경망을 nn.Sequential 클래스를 활용하여 정의한 모습이다.
# 단순히 원하는 연산을 수행할 내부 모듈의 nn.Sequential을 생성할 때, 순차적으로 넣어주는 것을 볼 수 있다.
# 당연한 얘기지만, 앞 쪽에 넣은 모듈들의 출력이 바로 뒷 모듈의 입력이 될 수 있도록 내부 모듈들 입출력 크기를 잘 적어 줘야 한다.
model = nn.Sequential(
    nn.Linear(x.size(-1), 3),
    nn.LeakyReLU(),
    nn.Linear(3, 3),
    nn.LeakyReLU(),
    nn.Linear(3, 3),
    nn.LeakyReLU(),
    nn.Linear(3, 3),
    nn.LeakyReLU(),
    nn.Linear(3, y.size(-1)),   
)
print(model)

In [ ]:
# 모델을 출력한 내용에서 아까와 다른 부분은 LeakyReLU를 사용하였다는 것과 하나의 LeakyReLU를 재활용하는 대신 매번 새로운 객체를 넣어준 것이다.
# 이렇게 선언한 모델의 가중치 파라미터들을 옵티마이저에 등록한다.
optimizer = optim.SGD(model.parameters(), lr=learning_rate) # model.paramters()는 모델의 모든 학습 가능한 파라미터를 반환한다.
# 즉, model.parameters()를 사용하여 optimizer를 초기화하면, optimizer는 모델의 모든 파라미터를 업데이트하게 된다.
# 이는 모델의 모든 파라미터를 한 번에 학습시키는데 유용하다.
# 따라서, model.paramters()는 모델이 nn.Module의 인스턴스일 때 자동으로 사용할 수 있다.
# 이 메소드는 모델의 구조(즉, 레이어의 구성 및 연결)에 따라 모델의 파라미터를 자동으로 추출한다.

In [ ]:
# 이제 모델을 학습할 준비가 끝났다.
# 본격적으로 심층신경망을 통해 회귀를 수행한다.
# n_epochs 만큼 for 반복문을 수행한다.
# 반복문 내부에는 피드포워드 및 손실 계산을 하고, 역전파와 경사하강을 수행하도록 구성되어 있다.
for i in range(n_epochs):
    y_hat = model(x)
    loss = F.mse_loss(y_hat, y) # MSE(Mean Squared Error) 평균 제곱 오차 손실 함수이다.
    # 예측값과 실제값 간의 차이를 제곱하여 평균을 계산한 후, 이를 반환한다.
    # 이 손실 함수는 회귀 모델에서 사용되며, 예측값과 실제값의 차이를 최소화하는 방향으로 모델을 학습시킨다.
    optimizer.zero_grad() # 모델의 가중치에 대한 미분값을 계산하기 위해 이전에 계산된 미분값을 초기화한다.
    # 파이토치에서 모델을 학습할 때 미분값을 계산하여 가중치를 업데이트한다.
    # 이때, 이전에 계산된 미분값이 남아있다면, 이전에 계산된 미분값이 더해져 오류가 발생할 수 있다.
    # 즉, optimizer.zeor_grad() 메서드를 호출하여, 이전에 계산된 미분값을 초기화해야 한다.
    loss.backward() # optimizer.zero_grad()를 호출하여, 모델의 가중치에 대한 미분값을 초기화한 후, loss.backward()를 이용하여 모델의 가중치에 대한 미분값을 계산한다.

    optimizer.step() # loss.backward()를 호출하여 모델의 가중치에 대한 미분값을 계산한 후, optimizer.step() 메서드를 호출하여 가중치를 업데이트한다.

    if(i + 1) % print_interval == 0:
        # print_interval은 파이토치에서 제공하는 train() 함수에서 사용되는 매개변수 중 하나이다.
        # print_interval은 로그를 출력하는 간격을 지정한다.
        print("Epoch %d: loss=%.4e" % (i + 1, loss))


In [ ]:
# print_interval 만큼 출력을 하도록 되어있으므로 이 코드를 실행하면 위와 같은 학습 결과를 얻을 수 있다.

# 결과 확인
# 앞에서 학습한 모델이 얼마나 잘 학습되었는지 시각화를 통해 확인한다.
df = pd.DataFrame(torch.cat([y, y_hat], dim=1).detach().numpy(), columns=["y", "y_hat"])

sns.pairplot(df, height=5)
plt.show()